In [2]:
from typing import List

import networkx as nx
from networkx import diameter, is_connected
from tqdm import tqdm

from rumor_centrality.evaluation import hop_distances
from rumor_centrality.graph_clustering import assign_all_nodes_cluster, visualise_cluster_graph, multiple_rumor_source_prediction
from rumor_centrality.graph_generator import small_world
from rumor_centrality.graph_simulations import si
# from rumor_centrality.experiment import multiple_sources_experiment
import rumor_centrality.jordan_center_alternative as jo
from rumor_centrality.rumor_detection import get_center_prediction

# Multiple rumor sources - Experiment execution
In this notebook we run the experiments to evaluate the performance of multiple source detection.

### Experiment Setup
Let's first define the most important parameters of our experiments, on which graphs to run them, and which metrics to use.

In [3]:
experiment_params = {
    "max_infected_nodes": [500, 1000],
    # "node_count": 3000,
    "infection_prob": 0.3,
    "num_infection_centers": [2, 3, 5, 7, 10],
    "exp_iterations": 10,
}

In [4]:
from functools import partial
from rumor_centrality.graph_generator import synthetic_internet, scale_free, us_power_grid, internet

# Available Graphs
# Making graphs bigger, so that infections make sense on it
graph_types = {
    # "synthetic_internet_100": lambda: synthetic_internet(1000),
    "synthetic_internet_10000": partial(synthetic_internet,10000),
    # "scale_free_100": lambda: nx.Graph(scale_free(1000)),
    "scale_free_10000": partial(scale_free, 10000),
    "us_power_grid": us_power_grid,
    "internet": internet,
}

In [5]:
metrics = {
    "rumor_centrality" : get_center_prediction,
    "jordan_centrality": jo.centers_by_jordan_center,
    "betweenness_centrality": jo.centers_by_betweenness_centrality,
    "distance_centrality": jo.centers_by_distance_centrality,
}

A small helper function to help with parallelization.

In [6]:
from itertools import product
from multiprocessing import Pool, TimeoutError
# from rumor_centrality.experiment import multiple_sources_experiment

def unpack_result(result):
    try:
        return_value = result.get(timeout=250)
    except TimeoutError:
        return_value = None

    return return_value

### Execution
Let's run the experiment.

In [22]:
from rumor_centrality.experiment import multiple_sources_experiment_metric

result_mult_metrics = []

num_infection_centers = experiment_params["num_infection_centers"]
max_infected_nodes = experiment_params["max_infected_nodes"]
infection_prob = experiment_params["infection_prob"]
exp_iterations = 50

for num_infection_center, max_inf_nodes, graph_name, metric_name in tqdm(list(product(num_infection_centers, max_infected_nodes, graph_types, metrics))):
    graph = graph_types[graph_name]
    metric = metrics[metric_name]

    with Pool(processes=10) as pool:
        multiple_results = [pool.apply_async(multiple_sources_experiment_metric, (
            num_infection_center,
            infection_prob,
            max_inf_nodes,
            graph,
            graph_name,
            metric,
            metric_name,
            metric_name != "rumor_centrality",
        )) for i in range(exp_iterations)]
        result_mult_metrics.extend([unpack_result(res) for res in multiple_results])

  4%|▎         | 1/28 [01:00<27:04, 60.16s/it]

0
found connected at 8
0
found connected at 14
0
found connected at 25
0
found connected at 3
0
found connected at 25
0
found connected at 17
0
found connected at 22
0
found connected at 5
0
found connected at 2
0
found connected at 7
0
found connected at 22
0
found connected at 20
0
found connected at 13
0
found connected at 7
0
found connected at 14
0
found connected at 9
0
found connected at 11
0
50
found connected at 66
0
found connected at 10
0
found connected at 17
0
found connected at 3
0
found connected at 6
0
50
found connected at 54
0
found connected at 1
0
found connected at 5
0
found connected at 3
0
found connected at 4
0
found connected at 3
0
found connected at 5
0
found connected at 8
0
found connected at 3
0
found connected at 3
0
found connected at 16
0
found connected at 29
0
found connected at 27
0
found connected at 6
0
found connected at 20
0
found connected at 14
0
found connected at 18
0
found connected at 11
0
found connected at 23
0
50
found connected at 59
0


  7%|▋         | 2/28 [02:09<28:16, 65.27s/it]

0
found connected at 29
0
found connected at 33
0
found connected at 3
0
found connected at 12
0
found connected at 35
0
found connected at 3
0
found connected at 13
0
found connected at 1
0
found connected at 1
0
found connected at 1
0
found connected at 1
0
found connected at 3
0
found connected at 1
0
found connected at 7
0
found connected at 19
0
found connected at 5
0
found connected at 17
0
found connected at 5
0
found connected at 2
0
found connected at 44
0
found connected at 13
0
found connected at 11
0
found connected at 7
0
found connected at 16
0
found connected at 5
0
found connected at 22
0
found connected at 41
0
found connected at 26
0
found connected at 8
0
50
found connected at 65
0
found connected at 23
0
found connected at 21
0
found connected at 12
0
found connected at 2
0
found connected at 17
0
found connected at 36
0
found connected at 1
0
50
found connected at 94
0
found connected at 1
0
found connected at 10
0
found connected at 27
0
found connected at 14
0
50

 11%|█         | 3/28 [03:02<24:54, 59.78s/it]

...

0
found connected at 6
0
found connected at 4
0
found connected at 6
0
found connected at 1
0
found connected at 6
0
found connected at 2
0
found connected at 5
0
found connected at 13
0
found connected at 2
0
found connected at 1
0
found connected at 8
0
found connected at 10
0
found connected at 1
0
found connected at 1
0
found connected at 2
0
found connected at 3
0
found connected at 3
0
found connected at 2
0
found connected at 2
0
found connected at 1
0
found connected at 4
0
found connected at 1
0
found connected at 1
0
found connected at 12
0
found connected at 7
0
found connected at 1
0
found connected at 8
0
found connected at 7
0
found connected at 4
0
found connected at 3
0
found connected at 1
0
found connected at 1
0
found connected at 1
0
found connected at 2
0
found connected at 10
0
found connected at 1
0
found connected at 3
0
found connected at 4
0
found connected at 1
0
found connected at 2
0
found connected at 19
0
found connected at 1
0
found connected at 8
0
foun

 96%|█████████▋| 27/28 [1:48:25<00:29, 29.81s/it]

0
found connected at 1
0
found connected at 1
0
found connected at 2
0
found connected at 1
0
found connected at 9
0
found connected at 1
0
found connected at 2
0
found connected at 7
0
found connected at 1
0
found connected at 11
0
found connected at 1
0
found connected at 1
0
found connected at 5
0
found connected at 2
0
found connected at 3
0
found connected at 1
0
found connected at 1
0
found connected at 5
0
found connected at 1
0
found connected at 1
0
found connected at 3
0
found connected at 5
0
found connected at 1
0
found connected at 10
0
found connected at 1
0
found connected at 8
0
found connected at 2
0
found connected at 4
0
found connected at 9
0
found connected at 3
0
found connected at 2
0
found connected at 3
0
found connected at 4
0
found connected at 3
0
found connected at 2
0
found connected at 1
0
found connected at 1
0
found connected at 2
0
found connected at 5
0
found connected at 1
0
found connected at 1
0
found connected at 1
0
found connected at 2
0
found c

100%|██████████| 28/28 [1:48:41<00:00, 232.91s/it]

0
found connected at 2
0
found connected at 3
0
found connected at 2
0
found connected at 5
0
found connected at 5
0
found connected at 2
0
found connected at 9
0
found connected at 3
0
found connected at 2
0
found connected at 2
0
found connected at 14
0
found connected at 2
0
found connected at 6
0
found connected at 2
0
found connected at 3
0
found connected at 8
0
found connected at 4
0
found connected at 6
0
found connected at 4
0
found connected at 1
0
found connected at 1
0
found connected at 4
0
found connected at 5
0
found connected at 1
0
found connected at 2
0
found connected at 1
0
found connected at 7
0
found connected at 3
0
found connected at 1
0
found connected at 10
0
found connected at 6
0
found connected at 4
0
found connected at 7
0
found connected at 1
0
found connected at 2
0
found connected at 1
0
found connected at 5
0
found connected at 8
0
found connected at 3
0
found connected at 3
0
found connected at 1
0
found connected at 1
0
found connected at 2
0
found c

### Saving the results
The calculation was expensive, let's save the results

In [24]:
from pathlib import Path
import pickle

with Path("multiple_centers_rumor_center_result_multiple_metrics.pickle").open("wb") as f:
    pickle.dump(result_mult_metrics, f)

Having a first look at the results

In [15]:
list(product(num_infection_centers, max_infected_nodes, graph_types, metrics))[100:128]

[(7, 500, 'scale_free_10000', 'rumor_centrality'),
 (7, 500, 'scale_free_10000', 'jordan_centrality'),
 (7, 500, 'scale_free_10000', 'betweenness_centrality'),
 (7, 500, 'scale_free_10000', 'distance_centrality'),
 (7, 500, 'us_power_grid', 'rumor_centrality'),
 (7, 500, 'us_power_grid', 'jordan_centrality'),
 (7, 500, 'us_power_grid', 'betweenness_centrality'),
 (7, 500, 'us_power_grid', 'distance_centrality'),
 (7, 500, 'internet', 'rumor_centrality'),
 (7, 500, 'internet', 'jordan_centrality'),
 (7, 500, 'internet', 'betweenness_centrality'),
 (7, 500, 'internet', 'distance_centrality'),
 (7, 1000, 'synthetic_internet_10000', 'rumor_centrality'),
 (7, 1000, 'synthetic_internet_10000', 'jordan_centrality'),
 (7, 1000, 'synthetic_internet_10000', 'betweenness_centrality'),
 (7, 1000, 'synthetic_internet_10000', 'distance_centrality'),
 (7, 1000, 'scale_free_10000', 'rumor_centrality'),
 (7, 1000, 'scale_free_10000', 'jordan_centrality'),
 (7, 1000, 'scale_free_10000', 'betweenness_cen

In [17]:
result_mult_metrics[-1]

{'hops': [1, 1, 1, 1, 2, 2, 2],
 'graph_normalized_hops': [0.2, 0.2, 0.2, 0.2, 0.4, 0.4, 0.4],
 'diameter': 5,
 'num_infection_centers': 7,
 'infection_prob': 0.3,
 'max_infected_nodes': 500,
 'predictions': [[208], [75, 350], [5], [796, 782], [1], [4], [1928]],
 'ground_truths': [1905, 3299, 3736, 6256, 6430, 7905, 8242],
 'graph': 'scale_free_10000',
 'metric': 'distance_centrality'}

### Experiment 2

To compare the values, we also run an experiment with the same method, only that we use only one infection cluster

In [7]:
from rumor_centrality.experiment import multiple_sources_experiment_metric

result_mult_metrics_single_cluster = []

num_infection_centers = [1]
max_infected_nodes = experiment_params["max_infected_nodes"]
infection_prob = experiment_params["infection_prob"]
exp_iterations = 50

for num_infection_center, max_inf_nodes, graph_name, metric_name in tqdm(list(product(num_infection_centers, max_infected_nodes, graph_types, metrics))):
    graph = graph_types[graph_name]
    metric = metrics[metric_name]

    with Pool(processes=10) as pool:
        multiple_results = [pool.apply_async(multiple_sources_experiment_metric, (
            num_infection_center,
            infection_prob,
            max_inf_nodes,
            graph,
            graph_name,
            metric,
            metric_name,
            metric_name != "rumor_centrality",
        )) for i in range(exp_iterations)]
        result_mult_metrics_single_cluster.extend([unpack_result(res) for res in multiple_results])



  3%|▎         | 1/32 [01:03<32:46, 63.45s/it]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


  6%|▋         | 2/32 [02:14<33:56, 67.88s/it]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


  9%|▉         | 3/32 [03:17<31:43, 65.64s/it]

...

 97%|█████████▋| 31/32 [13:05<00:18, 18.57s/it]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


100%|██████████| 32/32 [13:20<00:00, 25.02s/it]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


FileNotFoundError: [Errno 2] No such file or directory: 'multiple_centers_rumor_center_result_single_cluster.pickle'

Also saving the result...

In [10]:
with open("multiple_centers_rumor_center_result_single_cluster.pickle", "wb") as f:
    pickle.dump(result_mult_metrics_single_cluster, f)